In [5]:
import math, json, os, sys
from classification_models import ResNet18
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


# DATA_DIR = 'data'
TRAIN_DIR = "/home/jeniffer/abs/blurDetection/train"
VALID_DIR = "/home/jeniffer/abs/blurDetection/test"
SIZE = (224, 224)
BATCH_SIZE = 64

# Spatial Rich Models kernel initializer function
def srm(shape, dtype=None):
    data = np.load('/home/archimedes/abs/saravanan-rnd/forgery/custom/SRM_Kernels.npy')
    data = np.concatenate((data,data,data), axis = 2)
    return data

class CustomInitializer:
    def __call__(self, shape, dtype=None):
        return srm(shape, dtype=dtype)
    

            
            
if __name__ == "__main__":
    num_train_samples = sum([len(files) for r, d, files in os.walk(TRAIN_DIR)])
    num_valid_samples = sum([len(files) for r, d, files in os.walk(VALID_DIR)])

    num_train_steps = math.floor(num_train_samples/BATCH_SIZE)
    num_valid_steps = math.floor(num_valid_samples/BATCH_SIZE)

    gen = keras.preprocessing.image.ImageDataGenerator()
    val_gen = keras.preprocessing.image.ImageDataGenerator()

    batches = gen.flow_from_directory(TRAIN_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)
    val_batches = val_gen.flow_from_directory(VALID_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)
    classes = list(iter(batches.class_indices))
    model = ResNet18(input_shape=(224,224,3), weights='imagenet', include_top=True)
    #print(base_model.summary())
    model.layers.pop()
    model.layers.pop()
    print(model.summary())
    for layer in model.layers:
        layer.trainable=True
    last = model.layers[-1].output
    x = Dense(len(classes), activation="softmax")(last)
    finetuned_model = Model(model.input, x)
    finetuned_model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
#     x = keras.layers.GlobalAveragePooling2DgePooling2D((7,7))(base_model.output)
#     x = keras.layers.Dropout(0.3)(x)
#     x = Dense(len(classes), activation="softmax")(x)
#     finetuned_model = Model(base_model.input, x)
#     print(finetuned_model.summary())
# #     model.layers.pop()
#     for layer in finetuned_model.layers:
#         layer.trainable=True
#     last = model.layers[-1].output
#     x = Dense(len(classes), activation="softmax")(last)
    
    finetuned_model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    for c in batches.class_indices:
        classes[batches.class_indices[c]] = c
    finetuned_model.classes = classes

    early_stopping = EarlyStopping(patience=10)
    checkpointer = ModelCheckpoint('models/resnet18_blur_weights_{epoch:02d}.h5', verbose=1, save_best_only=True)

    finetuned_model.fit_generator(batches, steps_per_epoch=num_train_steps, epochs=20, callbacks=[early_stopping, checkpointer], validation_data=val_batches, validation_steps=num_valid_steps)
    finetuned_model.save('resnet18_blur_{epoch:02d}.h5')

Found 48000 images belonging to 2 classes.
Found 12000 images belonging to 2 classes.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 224, 224, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_19 (ZeroPadding2 (None, 230, 230, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 112, 112, 64) 9408        zero_padding2d_19[0][0]          
_______________________

Epoch 1/20
750/750 [==============================] - 600s 800ms/step - loss: 0.0174 - acc: 0.9936 - val_loss: 0.0041 - val_acc: 0.9983

Epoch 00001: val_loss improved from inf to 0.00410, saving model to resnet18_blur_weights_01.h5
Epoch 2/20
750/750 [==============================] - 590s 786ms/step - loss: 0.0056 - acc: 0.9983 - val_loss: 0.0018 - val_acc: 0.9992

Epoch 00002: val_loss improved from 0.00410 to 0.00179, saving model to resnet18_blur_weights_02.h5
Epoch 3/20
750/750 [==============================] - 591s 788ms/step - loss: 0.0043 - acc: 0.9988 - val_loss: 7.7613e-04 - val_acc: 0.9996

Epoch 00003: val_loss improved from 0.00179 to 0.00078, saving model to resnet18_blur_weights_03.h5
Epoch 4/20
750/750 [==============================] - 590s 787ms/step - loss: 0.0015 - acc: 0.9996 - val_loss: 3.6101e-04 - val_acc: 0.9999

Epoch 00004: val_loss improved from 0.00078 to 0.00036, saving model to resnet18_blur_weights_04.h5
Epoch 5/20
750/750 [============================